# Using the NCMATComposer to create new materials.

This notebook describes how one can use the NCMATComposer helper class from the NCrystal Python modules to create [[NCMAT data|NCMAT-format.md]] representing a given material to be used with NCrystal. This can either be done by directly inputting all desired material data directly, or by (fully or partially) using data from existing sources: NCMAT data, NCrystal.Info objects, NCrystal cfg-strings, CIF files, Quantum Espresso output, and many other files through [ASE](https://wiki.fysik.dtu.dk/ase/). Rather than being a complete API reference, the present page will simply present a range of examples.

## Preamble ##
Fixing dependencies + a few optional lines for nicer embedded plots.

In [ ]:
import pathlib
import os
if 'TSL_SCHOOL_DIR' in os.environ:
    if any( (p/".git").is_dir() for p in (pathlib.Path(".").absolute().resolve()/"dummy").parents ):
        raise RuntimeError('Please copy notebook to a work directory')

In [ ]:
#Uncomment to get dependencies via pip: !pip install --quiet ipympl matplotlib spglib

In [ ]:
%matplotlib ipympl

In [ ]:
import matplotlib
matplotlib.rcParams.update({"figure.figsize":(6.4*0.5,4.8*0.5),"figure.dpi":150,'font.size':5})

In [ ]:
%%html
<style>div.jupyter-widgets.widget-label {display: none;}</style>

Always import NCrystal of course:

In [ ]:
import NCrystal as NC
assert NC.version_num >=  3005081
#NC.test() #< quick unit test that installation works!

## Basic usage ##
For the most basic examples, we will keep things simple and look at how to construct both a simple gaseous material and a very simple crystalline material. We will in all cases use the `NCMATComposer` class from the NCrystal Python API to do this, although we could have also just written [NCMAT](https://github.com/mctools/ncrystal/wiki/NCMAT-format) text data directly, if we would have preferred this.

### A simple gaseous material
To start with, we can define a material described as a free gas of non-interacting atoms. For gasses, composing it like  We do this by creating an empty `NCMATComposer` instance, and then filling in dynamic information ("dyninfo") as appropriate:

*Small note: if you actually need gas mixtures in your simulations, you most likely do not want to create them using the NCMATComposer interface, since there is an alternative which is a lot easier and has more features: Simply use a special cfg-string like `"gasmix::0.7xCO2+0.3xAr/2bar/0.001relhumidity"` (more details on https://github.com/mctools/ncrystal/wiki/Announcement-Release3.2.0).*

In [ ]:
import NCrystal as NC
c_gas = NC.NCMATComposer()
c_gas.set_dyninfo_freegas( 'He', fraction = 0.91 )
c_gas.set_dyninfo_freegas( 'Ne', fraction = 0.09 )
c_gas.set_state_of_matter( 'gas' )#to be expressive, this line not strictly needed
c_gas.set_density(0.233642,'kg/m3')

Note that the entire NCrystal Python API includes documentation, so you can always do:

In [ ]:
# help( c_gas ) or help (NC.NCMATComposer) to show all methods of the NCMATComposer
help(c_gas.set_density) #<--- get info about a particular method

Now, let us see what the material we have defined in our NCMATComposer instance looks like. First let us inspect the resulting NCMAT data directly:

In [ ]:
print( c_gas.create_ncmat() )

That seems OK (although for such a simple material it would not have been hard to hand-write the NCMAT data!). For reference it should be noted that the reference documentation of the NCMAT format can be found at https://github.com/mctools/ncrystal/wiki/NCMAT-format

Instead of merely inspecting the NCMAT data, we can try to see how the material looks after being loaded with NCrystal:

In [ ]:
c_gas.inspect() #use c.plot_xsect() instead to avoid printouts and only see the plot

Now, assuming we are happy with the material, we can store it so it can be used in simulations. We can for instance do:

In [ ]:
c_gas.write('mysillygas.ncmat')

We can verify that the file was indeed written:

In [ ]:
import pathlib
print(pathlib.Path('mysillygas.ncmat').read_text())

And we can use it in NCrystal cfg-strings as usual (notice that just for fun we change the temperature when loading the material - refer to https://github.com/mctools/ncrystal/wiki/CfgRefDoc for details about the available cfg parameters):

In [ ]:
scatter = NC.createScatter("mysillygas.ncmat;temp=50K")
scatter.dump()

If you don't need a physical file, but only need the material for usage in the current process, you can simply register the file with a virtual name in memory instead:

In [ ]:
c_gas.register_as('mysillygas2.ncmat')

This created no actual file on the system, but NCrystal keeps it as a virtual file in memory:

In [ ]:
scatter = NC.createScatter("mysillygas2.ncmat;temp=50K")
scatter.dump()

*Small note: if you actually need gas mixtures in your simulations, you most likely do not want to create them using the NCMATComposer interface, since there is an alternative which is a lot easier and has more features: Simply use a special cfg-string like `"gasmix::0.7xCO2+0.3xAr/2bar/0.001relhumidity"` (more details on https://github.com/mctools/ncrystal/wiki/Announcement-Release3.2.0).*

### A simple crystalline material
Let us proceed to add a slightly more advanced material, one with a crystal structure. Albeit a simple one, that we can add by hand. We will add FCC aluminium, in which Al atoms sits on the face-centers and corners of a cubic unit cell of edge length 4.05Å:

In [ ]:
import NCrystal as NC
c_Al = NC.NCMATComposer()
c_Al.set_cellsg_cubic( 4.05 )#here we could also have provided spacegroup=225, but we will autodetect it instead below
c_Al.set_atompos( [ ('Al',0,0,0),
                 ('Al',0,1/2,1/2),
                 ('Al',1/2,0,1/2),
                 ('Al',1/2,1/2,0)])

This fixes the structure, and therefore also density and composition of the material. However, we are missing the material dynamics, which is a prerequisite for modelling any kind of scattering. As a quick workaround we can simply indicate that we are OK by using an idealised quadratic phonon spectrum for a Debye temperature of 300K:

In [ ]:
c_Al.allow_fallback_dyninfo()

In principle the material is now OK, however as a precaution against creating NCMAT data files with misleading or incorrect crystal structure, NCrystal will by default always use `spglib` ("space group library") to try to verify that the structure matches the provided spacegroup. Since we did *not* provide a spacegroup, this will fail:

In [ ]:
try:
    c_Al.create_ncmat()
except RuntimeError as e:
    print('FAILED:',e)

We can of course just tell NCrystal to forego this verification (**but do not do this!!!**):

In [ ]:
print( c_Al(verify_crystal_structure=False) )

Instead we can use `spglib` to detect the spacegroup from the symmetries given by our unit cell and atom positions:

In [ ]:
c_Al.refine_crystal_structure()

And now the material has been updated to include the spacegroup (which should be 225 for aluminium):

In [ ]:
print( c_Al() )

Hurray, indeed the spacegroup was detected as 225 as it should! For fun, you can go up and edit the atom positions in the `set_atompos(..)` call by changing the `'Al'` at `0,0,0` to `'Cr'`, run the cells again, and see how the detected spacegroup changes to 221: you broke some symmetries by having one of the atoms being of a different type, so the spacegroup changed.

Clearly the approach of manually adding atom positions is too cumbersome and error-prone for more complicated crystal structures, so we will see in the next Notebook how one can extract them from structures in existing files (esp. CIF files) and databases. For these cases, the crystal structure verification which was a bit annoying for our simple hand-written crystal, becomes very important since not all crystal definitions out there *in the wild* are entirely self consistent.

Finally, it should be mentioned that if you have knowledge of atomic displacements, Debye temperatures, or (best) phonon VDOS curves for the atoms in the material, you should absolutely add them as well with some sort of call like `c.set_dyninfo_xxx('Al',...)`. Refer to the next section, for more details about how to do that. The only thing to mention here, is that the `c.set_dyninfo_xxx('Al',...)` calls do not have to include a `fraction=...` parameter for crystalline materials, since the fractions can be inferred from the list of atomic positions.

We can of course again *inspect* the material we just composed:

In [ ]:
c_Al.inspect()

## Specifying dynamic information
Whether or not a material is crystalline, the dynamics of the contained atoms always has a direct connection to all modes of scattering of thermal neutrons on the material. We already saw above how one can mark the dynamics of a particular constituent to be modelled with a free gas model, using `c.set_dyninfo_freegas(...)`. Such a free-gas model is not in general appropriate for a description of a solid, where the atoms are after all bound rather than free. For completeness, NCrystal currently (v3.6) supports the following forms of dynamics information:

* **Free gas**: Model atoms according to a free-gas model. This is mainly intended for gasses and as a fall-back solution for less important components in liquids (e.g. O in H2O, since neutron scattering is dominated by scattering on H).
* **Scattering kernel**: Tabulated S(α,β) function (same as S(q,ω)), suitable for inelastic scattering in liquids.
* **Phonon DOS (VDOS) curve**, describing the frequency spectrum of oscillations of a particular component. This is by NCrystal both in order to create a scattering-kernel for modelling of inelastic scattering, and for determining the Debye-Waller factors needed for modelling of elastic scattering.
* **A Debye temperature**. This is used by NCrystal to create an idealised Phonon DOS curve, which is then used as in the previous item.
* **A mean-square-displacement (a.k.a. "Uiso")** of atoms at a given temperature. This is converted into a Debye temperature by the NCMATComposer.
* For completeness (for debugging purposes only!): There is also a "sterile" model, which can be used in gasses or liquids to remove the contribution to neutron scattering by a given material component.

### Example amorphous solid
As an example, here is how to setup an amorphous solid (a crude model of polyethylene, "PE") where all we know is the material composition, density, and mean-squared atomic displacements at a given temperature (here 50K):

In [ ]:
import NCrystal as NC
c_PE = NC.NCMATComposer()
help(c_PE.set_dyninfo_msd)

In [ ]:
c_PE.set_dyninfo_msd('H',msd=0.022,temperature=50,fraction=2/3)
c_PE.set_dyninfo_msd('C',msd=0.0076,temperature=50,fraction=1/3)
c_PE.set_density(0.92,'g/cm3')
print(c_PE.create_ncmat())

We will return to a discussion of atomic displacements and Debye temperatures shortly, but first let us check our material:

In [ ]:
c_PE.inspect()

#### Interlude: Debye model (debye_temp, atomic displacements, and idealised DOS curves)
You can use the following widget to investigate the relationship between Debye temperature and atomic displacements, and even to see the corresponding simplistic phonon DOS curve. Note that the numerical model used to correlate atomic displacements and Debye temperature is discussed in section 2.5 of the original NCrystal paper (https://doi.org/10.1016/j.cpc.2019.07.015).

In [ ]:
from ipywidgets import interact
@interact(debye_temp=(50.0,2000.0),atom_mass=(1.008,250.0),temperature=(1.0,2000),show_dos=['hide','unit THz','unit 1/cm','unit meV'])
def show_msd(debye_temp = 300,atom_mass=12.0,temperature=300.0,show_dos='hide'):
    msd=NC.debyeIsotropicMSD(debye_temperature=debye_temp,mass=atom_mass,temperature=temperature)
    print(f'\nDisplacement (δ² a.k.a. Uiso a.k.a. "msd" ): {msd:g}Å² = ({np.sqrt(msd):g}Å)²')
    print(f'Debye energy (k_boltzmann*debye_temp)      : {NC.constant_boltzmann*debye_temp*1000.0} meV\n')
    if show_dos.startswith('unit '):
        NCrystal.plot.plot_vdos(debye_temp,labelfct=lambda x : f'TDebye={debye_temp}K',unit=show_dos[5:])

### Direct scattering kernels (mostly for liquids)
It is of course also possible in NCrystal to add externally available scattering kernels to materials. However, there are several complications one needs to be aware of:

1. Such pre-calculated kernels will only be valid for a specific temperature, and attempting to modify the temperature to a different value (i.e. `"mymat.ncmat;temp=200K"`) will result in an error.

2. Such pre-calculated kernels are large. For that reason, we currently (v3.6) only ship two materials in our standard data library, namely room temperature water and heavy water. We do that for the simple reason, that our VDOS expansion code currently only supports solids, and not liquids. For other temperatures, one can find and download data files from the `validated` subfolder at: https://github.com/mctools/ncrystal-extra/tree/master/data . In the same repository, one can also find other materials converted from the ENDF v8 release, however one should most likely be very careful when using these, since they were not manually validated and at the very least lacks proper densities and crystal structures.

3. It is currently not possible for NCrystal to estimate atomic displacements from pre-calculated kernels, so materials with these will only provide inelastic scattering. Again, this is mostly suitable for liquids.

As an example, here is a quick view of the scattering kernel for oxygen in room temperature heavy water (note the need to tune clim/xlim/ylim keywords in order to get a reasonable plot):

In [ ]:
import NCrystal as NC
info_D2O = NC.createInfo('LiquidHeavyWaterD2O_T293.6K.ncmat')
di = info_D2O.findDynInfo('O')
di.plot_knl(clim=(0,1.5),xlim=(0,25),ylim=(-4,2),phasespace_curves=[NC.wl2ekin(1.8)])

**NB: Point out different conventions, S(q,w), downscattering up vs. down***

### Getting atomic displacement from VDOS

When a phonon VDOS curve is available, it is possible to estimate the mean-squared atomic displacements at a given temperature by performing a numerical integral over the DOS curve multiplied by a function $f(E)=\frac{1}{E\tanh(E/2kT)}$. 

Now, $E\tanh(E/2kT)$ tends to 0 like $E^2$, so in order for the integral to be well-formed, it is important that the DOS also tends to 0 like at least $E^2$ near $E=0$. This assumption is always required for phonon curves in NCrystal, and we will go into more details about that in another notebook.

The important point is: A phonon VDOS curve is sufficient to provide not only inelastic scattering physics in solids, but also the Debye-Waller factors needed to model elastic physics. Specifically NCrystal use these Debye-waller factors to model both incoherent-elastic components, as well as coherent-elastic components (incl. Bragg diffraction in case of a crystalline material).

## Comparing our simple Al+PE with more realistic materials:
Now, by not specifying full phonon DOS curves for our simplistic aluminium and polyethylene materials, we are of course loosing a lot of realism. We can compare the resulting inelastic cross section arising from our crude VDOS curves in `c_Al` and `c_PE`, with those from more realistic materials defined in the NCrystal std library:

In [ ]:
import NCrystal.plot
NCrystal.plot.plot_xsects('stdlib::Al_sg225.ncmat',c_Al,extra_cfg="comp=inelas")
NCrystal.plot.plot_xsects('stdlib::Polyethylene_CH2.ncmat',c_PE,extra_cfg="comp=inelas")

For aluminium the crude VDOS does relatively well, while the same is not the case for polyethylene. We can begin to understand why, if we look at the VDOS curves:

In [ ]:
import NCrystal.plot
info_stdAl = NC.createInfo('stdlib::Al_sg225.ncmat')
info_stdPE = NC.createInfo('stdlib::Polyethylene_CH2.ncmat')
NCrystal.plot.plot_vdos(c_PE.load().info.findDynInfo('H'),info_stdPE.findDynInfo('H'))
NCrystal.plot.plot_vdos(c_Al.load().info.findDynInfo('Al'),info_stdAl.findDynInfo('Al'))

Remember that the `TDebye=300K` for Al was simply a hard-coded fallback value (a value around `412K` would have been better). We will return to the connection between VDOS curves, scattering kernels, and neutron cross sections in the next notebook, as well as discussing how to acquire and-process such curves.

## Atom labels and compositions
In all the previous examples, all of the components like `"Al"`, `"C"`, `"H`", etc. we have added to materials in `NCMATComposer`, have been using labels which also happen to be names of elements of the periodic table. For that reason, we did not further need to define the actual atomic compositions of the individual components. However, we certainly can do so when needed. Additionally, in some materials the same element can appear in different roles - for instance imagine a monoatomic crystalline material where not all positions in the crystal unit cell are associated with the same average atomic displacement. In the following, we will show a few examples of this.

First, here is another go at a crude aluminium crystal, this time with a bit of chromium contaminating the structure:

In [ ]:
import NCrystal as NC
c_Al2 = NC.NCMATComposer()
c_Al2.set_cellsg_cubic( 4.05, spacegroup=225 )
c_Al2.set_atompos( [('Al',0,0,0),('Al',0,1/2,1/2),('Al',1/2,0,1/2),('Al',1/2,1/2,0)])
c_Al2.allow_fallback_dyninfo()
c_Al2.set_composition('Al','0.99 Al 0.01 Cr')
print(c_Al2())

The format used in the call to `.set_composition` can be very flexible, as described in the help:

In [ ]:
help(c_Al2.set_composition)

If we call `.set_composition`, it doesn't matter if we used a label like `"Al"` which is an element, or something else (`"atom1"`, `"my_Al"`, ...) -- the `.set_composition` call takes precedence. Here is an material where some aluminium atoms have different atomic displacements (hence the spacegroup is no longer 225):

In [ ]:
import NCrystal as NC
c_Al3 = NC.NCMATComposer()
c_Al3.set_cellsg_cubic( 4.05 )
c_Al3.set_atompos( [('tight_atom',0,0,0),('loose_atom',0,1/2,1/2),('loose_atom',1/2,0,1/2),('loose_atom',1/2,1/2,0)])
c_Al3.set_dyninfo_msd('tight_atom',msd=0.005, temperature=200)
c_Al3.set_dyninfo_msd('loose_atom',msd=0.02, temperature=200)
c_Al3.set_composition('tight_atom','Al')
c_Al3.set_composition('loose_atom','Al')
c_Al3.refine_crystal_structure()#Detect spacegroup
print(c_Al3())

## Adding secondary phases (+SANS)
bla bla... point out ncmat composer can start from cfgstrings

In [ ]:
c=NC.NCMATComposer('Al_sg225.ncmat')
c.add_secondary_phase(0.01,'void.ncmat')
c.add_hard_sphere_sans_model(50)
print(c())
c.inspect(mode='ekin')

## Modifying atom data like scattering lengths.
It should for completeness be mentioned that one can also set atomic properties like masses or scattering lengths. In practice, this mostly comes in handy if needing to use an element or isotope for which there is no well established data. The data can be provided using the `.update_atomdb` method:

In [ ]:
import NCrystal as NC
c_u = NC.NCMATComposer("solid::U/1gcm3")#note: can also init from a cfg-string!
c_u.set_composition('U','U232')
try:
    print(c_u.load())
except NC.NCBadInput as e:
    print('ERROR:',e)

In [ ]:
c_u.update_atomdb('U232','232.04u -12.3fm 4.5b 6.789b')#NB: dummy data values!
#alternative syntax: 
c_u.update_atomdb('U232',mass=232.04, coh_scat_len=-12.3, incoh_xs=4.5, abs_xs=6.789)
c_u.plot_xsect()

As mentioned already in another notebook, you can of course also inspect the values in NCrystal's built-in database:

In [ ]:
NC.atomDB('B10')#returns AtomData object

And see the entire database:

In [ ]:
for e in NC.iterateAtomDB():
    print(e)